In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load .env file using ipython
%dotenv ../.env

UsageError: Line magic function `%dotenv` not found.


In [3]:
import os

openai_api_key = os.getenv('openai_api_key')

In [4]:
from forvo_api import get_pronounciation
from langchain.llms import OpenAI
from langchain import PromptTemplate

In [5]:
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

word = input('Enter word: ')

In [6]:
# Notice "location" below, that is a placeholder for another value later
template = """
Please create two sentences in modern conversational Japanese that include the word/phrase 「{look_up_word}」 based on the following instructions:

You will provide a word or phrase, and I will create two realistic example sentences that may include verbs in either polite or dictionary forms. The sentences will be set in various everyday situations in modern Japan, such as at home, school, the store, a party, a restaurant, work between co-workers, on a date, a park, a train station, etc. Each sentence will be written in a way that can be understood by a student with proficiency at an N4 or N3 level and may include a proper Japanese first name or surname where appropriate. The sentences may be in a different tense, such as present, past, potential, te-form, subjective, or causative verb forms, and may be in either positive or negative form. The honorific or humble tense will not be used. The response will include an English translation and a note of the tense and grammar points used but absolutely do not include any Romanji please. The tone will be realistic and appropriate for a general audience. Thank you.
"""


In [8]:
prompt = PromptTemplate(
    input_variables=["look_up_word"],
    template=template,
)

get_pronounciation(word)
final_prompt = prompt.format(look_up_word=word) # keyword is 'word' which is defined in PromptTemplate Class

print (f"Final Prompt: {final_prompt}")
print ("-----------")
print (f"LLM Output: {llm(final_prompt)}")


Final Prompt: 
Please create two sentences in modern conversational Japanese that include the word/phrase 「猫」 based on the following instructions:

You will provide a word or phrase, and I will create two realistic example sentences that may include verbs in either polite or dictionary forms. The sentences will be set in various everyday situations in modern Japan, such as at home, school, the store, a party, a restaurant, work between co-workers, on a date, a park, a train station, etc. Each sentence will be written in a way that can be understood by a student with proficiency at an N4 or N3 level and may include a proper Japanese first name or surname where appropriate. The sentences may be in a different tense, such as present, past, potential, te-form, subjective, or causative verb forms, and may be in either positive or negative form. The honorific or humble tense will not be used. The response will include an English translation and a note of the tense and grammar points used but